In [1]:
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
url = 'https://raw.githubusercontent.com/tomwhite/covid-19-uk-data/master/data/covid-19-indicators-uk.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [3]:
#format as date
df['Date']=pd.to_datetime(df.Date)
#sort by date
df.sort_values(by='Date')

,Date,Country,Indicator,Value
0,2020-01-24,Scotland,ConfirmedCases,0
1,2020-01-24,Scotland,Deaths,0
2,2020-01-24,Scotland,Tests,2
7,2020-01-25,UK,Deaths,0
6,2020-01-25,UK,ConfirmedCases,0
...,...,...,...,...
692,2020-04-06,England,Deaths,4897
691,2020-04-06,England,ConfirmedCases,42990
702,2020-04-06,Wales,ConfirmedCases,3499
696,2020-04-06,Scotland,ConfirmedCases,3961


In [4]:
Deaths=df[df['Indicator']=="Deaths"]

In [5]:
Deaths

,Date,Country,Indicator,Value
1,2020-01-24,Scotland,Deaths,0
4,2020-01-25,Scotland,Deaths,0
7,2020-01-25,UK,Deaths,0
10,2020-01-26,Scotland,Deaths,0
13,2020-01-26,UK,Deaths,0
...,...,...,...,...
692,2020-04-06,England,Deaths,4897
694,2020-04-06,Northern Ireland,Deaths,70
697,2020-04-06,Scotland,Deaths,222
700,2020-04-06,UK,Deaths,5373


In [7]:
#Find date of first death
firstDeath={}


for index, row in Deaths.iterrows():
    if row.Value>0:
#         print(row.Area,row.Date)
        if row.Country not in firstDeath.keys():
            firstDeath[row.Country] = {'Country':row.Country,"firstDate":row.Date}


In [8]:
firstDeath

{'England': {'Country': 'England',
  'firstDate': Timestamp('2020-03-06 00:00:00')},
 'UK': {'Country': 'UK', 'firstDate': Timestamp('2020-03-06 00:00:00')},
 'Scotland': {'Country': 'Scotland',
  'firstDate': Timestamp('2020-03-14 00:00:00')},
 'Wales': {'Country': 'Wales', 'firstDate': Timestamp('2020-03-16 00:00:00')},
 'Northern Ireland': {'Country': 'Northern Ireland',
  'firstDate': Timestamp('2020-03-19 00:00:00')}}

In [10]:
#time since first death
theseAreas=[]

for index, row in Deaths.iterrows():
    if row.Country in firstDeath.keys():
#         print(row.Date-areaWithMoreThan10[row.Area]['firstDate'])
        if row.Date-firstDeath[row.Country]['firstDate'] >= datetime.timedelta(days = 0):
            theseAreas.append(row.Date-firstDeath[row.Country]['firstDate'])
        else:
            theseAreas.append(None)
    else:
        theseAreas.append(None)

Deaths['timeSince1']=theseAreas        

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
Deaths=Deaths[~pd.isnull(Deaths.timeSince1)]
#just get data where cases have timeSince1

In [27]:
firstDeath["England"]["firstDate"]


Timestamp('2020-03-06 00:00:00')

In [65]:
DeltaDeaths=[]

for index,row in Deaths.iterrows():
    if(row.Date!=firstDeath[row.Country]['firstDate']):
#         print(row.Date)
        print(row.Value-Deaths[(Deaths.Date==(row.Date-pd.Timedelta(days=1)))&(Deaths.Country==row.Country)].Value.iloc[0])
        DeltaDeaths.append(row.Value-Deaths[(Deaths.Date==(row.Date-pd.Timedelta(days=1)))&(Deaths.Country==row.Country)].Value.iloc[0])
    else:
        DeltaDeaths.append(None)
        
Deaths['ChangeinDeaths']=DeltaDeaths

1
1
0
0
1
1
3
3
0
0
2
2
2
2
10
11
14
0
14
19
0
20
14
1
16
1
31
1
32
0
37
3
41
0
32
0
0
33
1
55
0
1
56
0
35
1
3
48
9
46
0
4
54
4
82
2
2
87
1
27
3
6
41
5
103
3
3
115
6
164
3
8
181
6
247
2
7
260
4
199
6
1
209
10
159
1
6
180
14
367
6
22
381
7
515
2
28
563
29
532
6
29
569
19
604
12
46
684
24
637
8
46
708
13
555
7
2
621
12
403
7
2
439
27


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [70]:
lines=Deaths.pivot(index='Country',values='ChangeinDeaths',columns='timeSince1').transpose()

In [71]:
lines

Country,England,Northern Ireland,Scotland,UK,Wales
timeSince1,,,,,
0 days,NaN,NaN,NaN,NaN,NaN
1 days,1.0,0.0,0.0,1.0,1.0
2 days,0.0,0.0,0.0,0.0,0.0
3 days,1.0,1.0,1.0,1.0,0.0
4 days,3.0,0.0,1.0,3.0,1.0
5 days,0.0,2.0,3.0,0.0,0.0
6 days,2.0,3.0,0.0,2.0,9.0
7 days,2.0,3.0,1.0,2.0,4.0
8 days,10.0,3.0,3.0,11.0,1.0


In [72]:
lines.to_csv('DailyDeaths.csv', index = False)